In [1]:
#import local files to colab
from google.colab import files
uploaded = files.upload()

Saving users.json to users (3).json
Saving orders.csv to orders (3).csv
Saving restaurants.sql to restaurants (3).sql


In [2]:
#import required libraries
import pandas as pd
import sqlite3

In [3]:
#load transactional order data
orders = pd.read_csv('orders.csv')
#check data
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [4]:
#load user master data
users = pd.read_json('users.json')
#check data
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [6]:
#create sqlite database connection
conn = sqlite3.connect('restaurants.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS restaurants")
#read sql file
with open ("restaurants.sql","r") as f:
  sql_script = f.read()
#execute script
conn.executescript(sql_script)
#load restaurants table in pandas
restaurants = pd.read_sql('SELECT * FROM restaurants', conn)
#check data
restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [7]:
#LEFT join for orders
orders_users=pd.merge(orders,users,on="user_id",how="left")
orders_users.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


In [8]:
#Merge restaurant data
final_df = pd.merge(orders_users,restaurants,on="restaurant_id",how="left",suffixes=("_order","_restaurant"))
final_df = final_df.drop(columns=['restaurant_name_order'])
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,name,city,membership,restaurant_name_restaurant,cuisine,rating
0,1,2508,450,18-02-2023,842.97,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [10]:
#save final dataset
final_df.to_csv("final_food_delivery_dataset.csv",index=False)
files.download("final_food_delivery_dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
#city with highest total revenue from gold membership
gold_orders = final_df[final_df["membership"]=="Gold"]
gold_city_revenue = gold_orders.groupby("city")["total_amount"].sum()
gold_city_revenue.idxmax()

'Chennai'

In [14]:
#cuisine with highest order avg value across all orders
cuisine_avg_order = final_df.groupby("cuisine")["total_amount"].mean()
cuisine_avg_order.idxmax()

'Mexican'

In [15]:
#distinct users paid more than 1000 in total orders
user_total_spend = final_df.groupby("user_id")["total_amount"].sum()
high_value_users = user_total_spend[user_total_spend>1000]
high_value_users.count()

np.int64(2544)

In [17]:
#restaurant rating range generated highest total revenue
final_df["rating_range"]=pd.cut(
    final_df["rating"],
    bins=[3.0,3.5,4.0,4.5,5.0],
    labels=["3.0-3.5","3.6-4.0","4.1-4.5","4.6-5.0"],
    include_lowest=True
)
rating_range_revenue = final_df.groupby("rating_range")["total_amount"].sum()
rating_range_revenue.idxmax()

/tmp/ipython-input-386826931.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_range_revenue = final_df.groupby("rating_range")["total_amount"].sum()


'4.6-5.0'

In [20]:
#city with highest avg order value (gold members)
gold_orders = final_df[final_df["membership"]=="Gold"]
city_avg_order = gold_orders.groupby("city")["total_amount"].mean()
city_avg_order
city_avg_order.idxmax()

'Chennai'

In [21]:
#cuisine with lowest number of distinct restaurants but still contributes
restaurants_per_cuisine = final_df.groupby("cuisine")["restaurant_id"].nunique()
revenue_per_cuisine = final_df.groupby("cuisine")["total_amount"].sum()
cuisine_analysis = pd.DataFrame({
    "restaurant_count": restaurants_per_cuisine,
    "total_revenue": revenue_per_cuisine
})
cuisine_analysis.sort_values(
    by=["restaurant_count", "total_revenue"],
    ascending=[True, False]
)

,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


In [22]:
#percentage of orders placed by gold members
total_orders = final_df.shape[0]
gold_orders_count = final_df[final_df["membership"]=="Gold"].shape[0]
gold_percentage = (gold_orders_count/total_orders)*100
round(gold_percentage)

50

In [24]:
#restaurant with highest order avg value but less than 20 orders
restaurant_order_count = final_df.groupby("restaurant_name_restaurant")["order_id"].count()
restaurant_avg_value = final_df.groupby("restaurant_name_restaurant")["total_amount"].mean()
restaurant_analysis = pd.DataFrame({
    "order_count": restaurant_order_count,
    "avg_order_value": restaurant_avg_value
})
filtered_restaurants = restaurant_analysis[
    restaurant_analysis["order_count"] < 20
]
filtered_restaurants.sort_values(
    by="avg_order_value",
    ascending=False
)


,order_count,avg_order_value
restaurant_name_restaurant,,
Restaurant_294,13,1040.222308
Restaurant_262,18,1029.473333
Restaurant_77,12,1029.180833
Restaurant_193,15,1026.306667
Restaurant_7,16,1002.140625
...,...,...
Restaurant_184,19,621.828947
Restaurant_498,18,596.815556
Restaurant_192,14,589.972857


In [30]:
#combination contributes most
combo_revenue = final_df.groupby(
    ["membership", "cuisine"]
)["total_amount"].sum()
combo_revenue.sort_values(ascending=False).index[1]


('Regular', 'Italian')

In [32]:
#quarter of year with highest revenue
final_df["order_date"] = pd.to_datetime(
    final_df["order_date"],
    errors="coerce"
)
final_df["quarter"] = final_df["order_date"].dt.quarter
quarter_revenue = final_df.groupby("quarter")["total_amount"].sum()
quarter_revenue
quarter_revenue.idxmax()



/tmp/ipython-input-3296011049.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(


np.int32(3)

In [33]:
#total orders placed by gold users
gold_orders_count = final_df[final_df["membership"]=="Gold"].shape[0]
gold_orders_count

4987

In [34]:
#total revenue from orders in Hyderabad
hyderabad_revenue = final_df[final_df["city"]=="Hyderabad"]["total_amount"].sum()
round(hyderabad_revenue)

1889367

In [35]:
#distinct users placed order
distinct_users = final_df["user_id"].nunique()
distinct_users

2883

In [38]:
#avg order value for gold members
gold_avg_order_value = final_df[final_df["membership"] == "Gold"]["total_amount"].mean()
round(gold_avg_order_value, 2)

np.float64(797.15)

In [39]:
#orders placed for restaurants with rating >4.5
high_rating_orders_count = final_df[final_df["rating"] >= 4.5].shape[0]
high_rating_orders_count


3374

In [40]:
#orders placed in top revenue city with gold members
gold_orders = final_df[final_df["membership"] == "Gold"]
gold_city_revenue = gold_orders.groupby("city")["total_amount"].sum()
top_gold_city = gold_city_revenue.idxmax()
top_gold_city
gold_top_city_orders = gold_orders[gold_orders["city"] == top_gold_city]
gold_top_city_orders.shape[0]


1337